# AIMS Data Platform - Exploration Notebook

This notebook demonstrates the federated data platform capabilities including:
- Incremental data loading
- Data quality validation with Great Expectations
- Data contracts enforcement
- Microsoft Fabric compatibility

## 1. Setup and Configuration

In [4]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd() / 'src'))

import pandas as pd
import pyarrow.parquet as pq
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

#from config.settings import settings
#from contracts.registry import ContractRegistry
# from ingestion.watermark_manager import WatermarkManager
# from ingestion.incremental_loader import IncrementalLoader

print("✅ Imports successful")

✅ Imports successful


In [5]:
# Initialize settings and ensure directories exist
settings.ensure_directories()

print(f"Source Data Path: {settings.source_data_path}")
print(f"Target Data Path: {settings.target_data_path}")
print(f"Watermark DB Path: {settings.watermark_db_path}")

NameError: name 'settings' is not defined

## 2. Explore Source Parquet Files

In [6]:
# List all parquet files in source directory
parquet_files = list(settings.source_data_path.glob('*.parquet'))
print(f"Found {len(parquet_files)} parquet files:")
for f in parquet_files[:10]:  # Show first 10
    print(f"  - {f.name}")

NameError: name 'settings' is not defined

In [ ]:
# Read and inspect first parquet file
if parquet_files:
    sample_file = parquet_files[0]
    df_sample = pd.read_parquet(sample_file)
    
    print(f"\n📊 Sample from {sample_file.name}:")
    print(f"Shape: {df_sample.shape}")
    print(f"\nColumns: {list(df_sample.columns)}")
    print(f"\nData types:\n{df_sample.dtypes}")
    print(f"\nFirst 5 rows:")
    display(df_sample.head())

In [ ]:
# Get schema information from parquet file
if parquet_files:
    parquet_file = pq.ParquetFile(parquet_files[0])
    schema = parquet_file.schema
    print("\n🔍 Parquet Schema:")
    print(schema)

## 3. Initialize Data Platform Components

In [ ]:
# Initialize Spark session
spark = (
    SparkSession.builder
    .appName("AIMS-DataPlatform")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

print(f"✅ Spark {spark.version} session created")

In [ ]:
# Initialize watermark manager
watermark_manager = WatermarkManager(settings.watermark_db_path)
print("✅ Watermark manager initialized")

In [ ]:
# Initialize contract registry
contract_registry_path = Path.cwd() / "src" / "contracts" / "schemas"
contract_registry = ContractRegistry(contract_registry_path)
print(f"✅ Contract registry initialized")
print(f"Available contracts: {contract_registry.list_contracts()}")

In [ ]:
# Initialize incremental loader
loader = IncrementalLoader(
    watermark_manager=watermark_manager,
    contract_registry=contract_registry,
    spark=spark
)
print("✅ Incremental loader initialized")

## 4. Data Contracts

In [ ]:
# View data contract
contract = contract_registry.get("aims_data")
if contract:
    print(f"📋 Contract: {contract.name} v{contract.version}")
    print(f"Description: {contract.description}")
    print(f"\nFields:")
    for field in contract.fields:
        nullable = "nullable" if field.nullable else "required"
        print(f"  - {field.name}: {field.data_type} ({nullable})")
    print(f"\nPartition Keys: {contract.partition_keys}")
    print(f"Primary Keys: {contract.primary_keys}")
else:
    print("⚠️ No contract found. You may need to adjust the schema file.")

## 5. Incremental Data Loading

In [ ]:
# Check current watermark
source_name = "aims_parquet_data"
current_watermark = watermark_manager.get_watermark(source_name)
print(f"Current watermark for '{source_name}': {current_watermark or 'None (first load)'}")

In [ ]:
# Example: Load data incrementally
# Note: Adjust the watermark_column to match your actual data
"""
result = loader.load_incremental(
    source_name="aims_parquet_data",
    source_path=settings.source_data_path,
    target_path=settings.target_data_path / "aims_delta",
    watermark_column="timestamp",  # Adjust based on your data
    contract_name="aims_data",
    partition_columns=["created_date"]
)

print("\n📊 Load Results:")
for key, value in result.items():
    print(f"  {key}: {value}")
"""
print("⚠️ Uncomment and adjust the code above to perform incremental load")

## 6. Query Delta Table

In [ ]:
# Read Delta table (if it exists)
delta_path = settings.target_data_path / "aims_delta"

if delta_path.exists():
    df_delta = spark.read.format("delta").load(str(delta_path))
    
    print(f"\n📊 Delta Table Statistics:")
    print(f"Total records: {df_delta.count()}")
    print(f"\nSchema:")
    df_delta.printSchema()
    
    print(f"\nSample data:")
    df_delta.show(5)
else:
    print("⚠️ Delta table not found. Run incremental load first.")

In [ ]:
# Query Delta table with Spark SQL
if delta_path.exists():
    df_delta.createOrReplaceTempView("aims_data")
    
    # Example queries
    print("\n📊 Data Summary:")
    spark.sql("""
        SELECT 
            COUNT(*) as total_records,
            COUNT(DISTINCT id) as unique_ids,
            MIN(timestamp) as earliest_timestamp,
            MAX(timestamp) as latest_timestamp
        FROM aims_data
    """).show()

## 7. Data Quality Checks

In [ ]:
# Basic data quality checks
if delta_path.exists():
    df = spark.read.format("delta").load(str(delta_path))
    
    print("\n🔍 Data Quality Checks:")
    
    # Check for nulls
    print("\nNull counts:")
    df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()
    
    # Check for duplicates (if primary keys exist)
    if contract and contract.primary_keys:
        duplicate_count = (
            df.groupBy(*contract.primary_keys)
            .count()
            .filter(F.col("count") > 1)
            .count()
        )
        print(f"\nDuplicate records (by primary keys): {duplicate_count}")

## 8. Export to Pandas for Analysis

In [ ]:
# Convert to Pandas for further analysis
if delta_path.exists():
    # Sample data to avoid memory issues
    df_pandas = df_delta.limit(10000).toPandas()
    
    print(f"\n📊 Pandas DataFrame:")
    print(f"Shape: {df_pandas.shape}")
    display(df_pandas.head())
    
    print(f"\nDescriptive statistics:")
    display(df_pandas.describe())

## 9. Cleanup

In [ ]:
# Stop Spark session
# spark.stop()
# print("✅ Spark session stopped")